### Notes
 -  Schaefer parcellation: The atlas divides the brain into 100 (or 200 etc.) ROIs. Each region (ROI) represents a group of voxels that are functionally connected. 
- The parcellation_nifti variable (atlas['maps']) contains a 3D brain image where each voxel has a label (1 to 100), indicating which ROI it belongs to.
- NiftiLabelsMasker applies the Schaefer atlas to the 4D fMRI image: Each voxel in the brain is associated with an ROI label (1 to 100), It computes the average BOLD signal within each ROI across time. The result is a 2D time series matrix with dimensions (t,num_ROIs), and the BOLD signals within each ROI are standardized using z-score normalization to ensure that they have a mean of 0 and a standard deviation of 1.

In [ ]:
import numpy as np
import nibabel as nib
from nilearn.input_data import NiftiLabelsMasker
from nilearn.datasets import fetch_atlas_schaefer_2018
import os

%cd /Users/xuenbei/Desktop/fyp/fyp/ds003059

cwd = os.getcwd()  # Get the current working directory (cwd)
subjects = [directory for directory in os.listdir(cwd) if ('sub' in directory and not '.txt' in directory)] 
print(subjects)

# The Schaefer atlas divides the brain into 100 distinct regions of interest (ROIs)

atlas = fetch_atlas_schaefer_2018(n_rois=100)  

# The `parcellation_nifti` object contains a NIfTI image file where each voxel is labeled with an ROI index (1 through 100).
parcellation_nifti = atlas['maps']

for subj in subjects:
    print(subj)
    plcb_path_1 = f'{subj}/ses-PLCB/func/{subj}_ses-PLCB_task-rest_run-02_bold.nii.gz'
    lsd_path_1 = f'{subj}/ses-LSD/func/{subj}_ses-LSD_task-rest_run-02_bold.nii.gz'
    paths = [plcb_path_1, lsd_path_1]
    
    # Loop over the file paths for each session
    for path in paths:
        try:
            func_nifti = nib.load(path) # loads the 4D functional MRI NIfTI image corresponding to a specific subject and session. The dimensions of this image are typically (x, y, z, t)
        except FileNotFoundError:
            print(f"File not found: {path}. Skipping.") # if a file is missing, skips
            continue
        
        # Mask the functional MRI data using the Schaefer parcellation
        # NiftiLabelsMasker: 
        # Applies the Schaefer atlas mask to the 4D fMRI data.
        # Extracts the average BOLD signal within each of the 100 regions (ROIs).
        # Standardizes the time series (z-score normalization).
        
        masker = NiftiLabelsMasker(labels_img=parcellation_nifti, standardize=True)
        time_series = masker.fit_transform(func_nifti)
        
        # Check and log the shape of the time series
        print(f"Extracted time series shape for {path}: {time_series.shape}")
        
        # Ensure time_series is valid, the result should be a 2D array (time points, num ROI)
        # If this value is 0, it means no regions were extracted.
        if len(time_series.shape) < 2 or time_series.shape[1] == 0:
            print(f"Invalid time series extracted for {path}. Skipping.")
            continue
        
        for region_index in range(time_series.shape[1]):  # Should only iterate from 0 to 99
            ts_for_region = time_series[:, region_index]
            
            if 'ses-PLCB' in path:
                file_name = f'/Users/xuenbei/Desktop/finalyearproject/extract_time_series/{subj}-PLCB-ROI{region_index}.txt'
            else:
                file_name = f'/Users/xuenbei/Desktop/finalyearproject/extract_time_series/{subj}-LSD-ROI{region_index}.txt'
            
            # Save time series 
            np.savetxt(file_name, ts_for_region, delimiter=', ')


# Verify the saved time series
%cd /Users/xuenbei/Desktop/fyp/fyp/extract_time_series/ROI

cwd = os.getcwd()  # Get the current working directory (cwd)
file_paths = [directory for directory in os.listdir(cwd) if directory.endswith('.txt')]  # Get all saved .txt files

# Process each saved time series file
for path in file_paths:
    time_series = np.loadtxt(path, delimiter=',')
    
    file_name = os.path.basename(path)  #  extracts the base name (i.e., the final part) of a file path in Python. e.g. 'sub-002-LSD-ROI10.txt'
    
    # Print the correct file name and shape of the time series
    if len(time_series.shape) == 1:  # Shape (time points,) expected for valid files
        print(f"Successfully loaded time series for {file_name}: {time_series.shape}")
    else:
        print(f"Unexpected shape for {file_name}: {time_series.shape}. Please check the file.")


### Checking Shaefer Parcellation

In [40]:
from nilearn.datasets import fetch_atlas_schaefer_2018
import nibabel as nib
import numpy as np

atlas = fetch_atlas_schaefer_2018(n_rois=100)

# Load and check the atlas ---
atlas_img = nib.load(atlas['maps'])
atlas_data = atlas_img.get_fdata()

# Access the path to the parcellation file containing the brain parcellation maps 
parcellation_path = atlas['maps']
print("Parcellation file path:", parcellation_path)

# Load the parcellation NIfTI image using nibabel 
parcellation_nifti = nib.load(parcellation_path)

print("Shape of the parcellation:", parcellation_nifti.shape)

# Count and print unique labels 
unique_labels = np.unique(atlas_data)
print(f"Number of unique regions in the atlas: {len(unique_labels)}")
print(f"Unique labels: {unique_labels}")

region_labels = atlas['labels']  # List of region names

# Print each index and its corresponding region label
for idx, label in enumerate(region_labels):
    print(f"ROI Index {idx}: {label}")


[get_dataset_dir] Dataset found in /Users/xuenbei/nilearn_data/schaefer_2018

Parcellation file path: /Users/xuenbei/nilearn_data/schaefer_2018/Schaefer2018_100Parcels_7Networks_order_FSLMNI152_1mm.nii.gz
Shape of the parcellation: (182, 218, 182)
Number of unique regions in the atlas: 101
Unique labels: [  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.
  14.  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.
  28.  29.  30.  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.  41.
  42.  43.  44.  45.  46.  47.  48.  49.  50.  51.  52.  53.  54.  55.
  56.  57.  58.  59.  60.  61.  62.  63.  64.  65.  66.  67.  68.  69.
  70.  71.  72.  73.  74.  75.  76.  77.  78.  79.  80.  81.  82.  83.
  84.  85.  86.  87.  88.  89.  90.  91.  92.  93.  94.  95.  96.  97.
  98.  99. 100.]
ROI Index 0: b'7Networks_LH_Vis_1'
ROI Index 1: b'7Networks_LH_Vis_2'
ROI Index 2: b'7Networks_LH_Vis_3'
ROI Index 3: b'7Networks_LH_Vis_4'
ROI Index 4: b'7Networks_LH_Vis_5'
ROI Index 5: b'7Networks_LH_Vis_6'
ROI Index 6: b'7Networks_LH_Vis_7'
ROI Index 7: 

1. Visual Network (Vis):
- ROI Index 0 to 9 and 50 to 59: These regions are part of the visual network in the left and right hemispheres (LH and RH). The visual network is primarily responsible for processing visual information.

2. Somatomotor Network (SomMot):
- ROI Index 9 to 14 and 58 to 63: This network is involved in motor control and sensation, which includes areas such as the somatosensory cortex. 

3. Dorsal Attention Network (DorsAttn):
- ROI Index 15 to 22 and 66 to 72: This network is involved in attentional control, particularly in tasks requiring focus and the integration of sensory information. 

4. Salience and Ventral Attention Networks (SalVentAttn):
- ROI Index 23 to 29 and 73 to 77: These regions are associated with detecting and responding to significant stimuli, including emotional and sensory information. 

5. Limbic Network (Limbic):
- ROI Index 30 to 32 and 78 to 79: These regions are associated with emotional processing, memory, and motivation. 

6. Cognitive Control Networks (Cont):
- ROI Index 33 to 47 and 80 to 88: These regions are involved in executive functions like decision-making, planning, and self-regulation. 

7. Default Mode Network (Default):
- ROI Index 37 to 49 and 89 to 99: The default mode network (DMN) is active when the brain is at rest or not focused on external tasks.

In [2]:
import nibabel as nib
import numpy as np

nii_path = '/Users/xuenbei/Desktop/finalyearproject/time_series/Schaefer2018_100Parcels.nii.gz'
img = nib.load(nii_path)

# Get the data as a NumPy array
data = img.get_fdata()

# Print some basic information
print("Shape of the data:", data.shape)
print("Data type:", data.dtype)
print("Affine matrix:\n", img.affine)

# Optionally print a slice (e.g., the middle slice)
import numpy as np
middle_slice = data.shape[2] // 2
print("Middle slice of the volume (Z = {}):".format(middle_slice))
print(data[:, :, middle_slice])

unique_parcels = np.unique(data)
print("Unique parcel IDs:", unique_parcels)

Shape of the data: (91, 109, 91)
Data type: float64
Affine matrix:
 [[  -2.    0.    0.   90.]
 [   0.    2.    0. -126.]
 [   0.    0.    2.  -72.]
 [   0.    0.    0.    1.]]
Middle slice of the volume (Z = 45):
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Unique parcel IDs: [  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.
  14.  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.
  28.  29.  30.  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.  41.
  42.  43.  44.  45.  46.  47.  48.  49.  50.  51.  52.  53.  54.  55.
  56.  57.  58.  59.  60.  61.  62.  63.  64.  65.  66.  67.  68.  69.
  70.  71.  72.  73.  74.  75.  76.  77.  78.  79.  80.  81.  82.  83.
  84.  85.  86.  87.  88.  89.  90.  91.  92.  93.  94.  95.  96.  97.
  98.  99. 100.]


In [4]:
import nibabel as nib

func_file = '/Users/xuenbei/Desktop/final_year_project/ds003059/sub-002/ses-PLCB/func/sub-002_ses-PLCB_task-rest_run-03_bold.nii.gz'
img = nib.load(func_file)
hdr = img.header
tr = hdr.get_zooms()[3]  # TR is the 4th value in the zooms
print("TR =", tr, "seconds")


TR = 2.0 seconds


In [1]:
import numpy as np
import nibabel as nib
from nilearn.input_data import NiftiLabelsMasker
import os

# Change to your data directory
%cd /Users/xuenbei/Desktop/finalyearproject/ds003059

cwd = os.getcwd()  # Get the current working directory
subjects = [directory for directory in os.listdir(cwd) if ('sub' in directory and not '.txt' in directory)]
print("Subjects found:", subjects)

# Load the local Schaefer atlas file (100 parcels)
parcellation_nifti = nib.load('/Users/xuenbei/Desktop/finalyearproject/time_series/Schaefer2018_100Parcels.nii.gz')

for subj in subjects:
    print(f"Processing subject: {subj}")
    plcb_path_1 = f'{subj}/ses-PLCB/func/{subj}_ses-PLCB_task-rest_run-02_bold.nii.gz'
    lsd_path_1 = f'{subj}/ses-LSD/func/{subj}_ses-LSD_task-rest_run-02_bold.nii.gz'
    paths = [plcb_path_1, lsd_path_1]
    
    for path in paths:
        try:
            func_nifti = nib.load(path)
        except FileNotFoundError:
            print(f"File not found: {path}. Skipping.")
            continue
        
        # Create masker with your local atlas
        masker = NiftiLabelsMasker(labels_img=parcellation_nifti, standardize=True)
        
        # Extract average time series for each ROI
        time_series = masker.fit_transform(func_nifti)
        
        print(f"Extracted time series shape for {path}: {time_series.shape}")
        
        if len(time_series.shape) < 2 or time_series.shape[1] == 0:
            print(f"Invalid time series extracted for {path}. Skipping.")
            continue
        
        # Save each ROI's time series to separate text files
        for region_index in range(time_series.shape[1]):
            ts_for_region = time_series[:, region_index]
            
            if 'ses-PLCB' in path:
                file_name = f'/Users/xuenbei/Desktop/finalyearproject/time_series1/{subj}-PLCB-ROI{region_index}.txt'
            else:
                file_name = f'/Users/xuenbei/Desktop/finalyearproject/time_series1/{subj}-LSD-ROI{region_index}.txt'
            
            np.savetxt(file_name, ts_for_region, delimiter=', ')
            

%cd /Users/xuenbei/Desktop/finalyearproject/time_series1

file_paths = [f for f in os.listdir() if f.endswith('.txt')]
print(f"Found {len(file_paths)} time series files.")

for path in file_paths:
    time_series = np.loadtxt(path, delimiter=',')
    file_name = os.path.basename(path)
    
    if len(time_series.shape) == 1:
        print(f"Successfully loaded time series for {file_name}: {time_series.shape}")
    else:
        print(f"Unexpected shape for {file_name}: {time_series.shape}. Please check the file.")


/Users/xuenbei/miniconda3/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/Users/xuenbei/Desktop/finalyearproject/ds003059
Subjects found: ['sub-019', 'sub-010', 'sub-017', 'sub-011', 'sub-018', 'sub-020', 'sub-002', 'sub-004', 'sub-003', 'sub-013', 'sub-012', 'sub-015', 'sub-006', 'sub-001', 'sub-009']
Processing subject: sub-019
Extracted time series shape for sub-019/ses-PLCB/func/sub-019_ses-PLCB_task-rest_run-02_bold.nii.gz: (217, 100)
Extracted time series shape for sub-019/ses-LSD/func/sub-019_ses-LSD_task-rest_run-02_bold.nii.gz: (217, 100)
Processing subject: sub-010
Extracted time series shape for sub-010/ses-PLCB/func/sub-010_ses-PLCB_task-rest_run-02_bold.nii.gz: (217, 100)
Extracted time series shape for sub-010/ses-LSD/func/sub-010_ses-LSD_task-rest_run-02_bold.nii.gz: (217, 100)
Processing subject: sub-017
Extracted time series shape for sub-017/ses-PLCB/func/sub-017_ses-PLCB_task-rest_run-02_bold.nii.gz: (217, 100)
Extracted time series shape for sub-017/ses-LSD/func/sub-017_ses-LSD_task-rest_run-02_bold.nii.gz: (217, 100)
Processing subject: